In [2]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
from scipy.stats import uniform,norm

Generate synthetic lightcurve with count rate being r'=r+gamma\*sigma 

where sigma is flux error from original lc.*

Code automatically scans nicer file directory for 9 digit obsid and computes on them


In [3]:
# ##########################################
# ####***********WRONG CODE*************####
############################################


# def generate_synthetic_light_curves(input_fits, output_folder, num_synthetic_curves):
#     # Extract observation ID from the input file path (assumes folder name is observation ID)
    
#     obs_id = input_fits.split('/')[1]

#     # Create a specific output directory for the synthetic light curves and best_fit_period.txt
#     output_dir = os.path.join(output_folder, obs_id)
#     os.makedirs(output_dir, exist_ok=True)

#     # Path for storing the best fit period file for the current obs_id
#     best_fit_period_file = os.path.join(output_dir, 'best_fit_period_samir_sir_suggestion2.txt')

#     # Open the input FITS file
#     with fits.open(input_fits) as hdul:
#         rate_data = hdul['RATE'].data
#         time = rate_data['TIME']
#         rate = rate_data['RATE']
#         error = rate_data['ERROR']

#         # Generate synthetic light curves and run efsearch
#         for i in range(num_synthetic_curves):

#             print("rate \n",rate[:10],'\nerror \n',error[:10])

#             synthetic_rate = np.zeros_like(rate)  # Initialize array for synthetic rate
#             random_array=np.random.uniform(0, 1, len(rate))
#             print("random number \n",random_array[:10])
#             # Save the random array to a text file
#             random_array_file = os.path.join(output_dir, f'random_array_{i+1}.txt')
#             np.savetxt(random_array_file, random_array)  # Save random array
            
#             gamma_j = 2.0 * random_array - 1
#             print(gamma_j[:10])
#             synthetic_rate= rate + gamma_j*error # Apply to each bin
#             print("synthetic rate \n",synthetic_rate[:10])
#             print(f"printed for {i}")
#             #synthetic_rate[synthetic_rate<0]=0 ## All bins also go negative
#             # Create a new synthetic FITS file
#             output_fits = os.path.join(output_dir, f'synthetic_light_curve_{i+1}.fits')

#             hdul_new = hdul.copy()
#             hdul_new['RATE'].data['RATE'] = synthetic_rate
#             print("old_rate\n",hdul['RATE'].data['RATE'][:10])
#             hdul_new.writeto(output_fits, overwrite=True)

In [4]:
def generate_synthetic_light_curves(input_fits, output_folder, num_synthetic_curves):
    # Extract observation ID from the input file path (assumes folder name is observation ID)
    obs_id = input_fits.split('/')[1]

    # Create a specific output directory for the synthetic light curves and best_fit_period.txt
    output_dir = os.path.join(output_folder, obs_id)
    os.makedirs(output_dir, exist_ok=True)

    # Path for storing the best fit period file for the current obs_id
    best_fit_period_file = os.path.join(output_dir, 'best_fit_period_1000.txt')

    # Open the input FITS file
    with fits.open(input_fits) as hdul:
        rate_data = hdul['RATE'].data
        time = rate_data['TIME']
        rate = rate_data['RATE']
        error = rate_data['ERROR']

        # Generate synthetic light curves and run efsearch
        for i in range(num_synthetic_curves):
            synthetic_rate = np.zeros_like(rate)  # Initialize array for synthetic rate
            random_array = np.random.uniform(0, 1, len(rate))
            # Save the random array to a text file
            # random_array_file = os.path.join(output_dir, f'random_array_{i+1}.txt')
            # np.savetxt(random_array_file, random_array)  # Save random array
            
            gamma_j = 2.0 * random_array - 1  
            synthetic_rate = rate + gamma_j * error  # Apply to each bin
            # synthetic_rate[synthetic_rate<0]=0 ## All bins also go negative
            
            # Create a new synthetic FITS file
            output_fits = os.path.join(output_dir, f'synthetic_light_curve_{i+1}.fits')

            # Copy the primary HDU (headers and data)
            primary_hdu = fits.PrimaryHDU(header=hdul[0].header, data=hdul[0].data)
            
            # Copy the RATE extension but modify the 'RATE' column
            new_rate_hdu = fits.BinTableHDU(data=rate_data.copy(), header=hdul['RATE'].header)
            new_rate_hdu.data['RATE'] = synthetic_rate  # Update synthetic rates
            
            # Preserve all extensions and headers
            new_hdul = fits.HDUList([primary_hdu, new_rate_hdu])

            # Write the synthetic FITS file
            new_hdul.writeto(output_fits, overwrite=True)

            #########Debugging###########
            # print("rate \n", rate[:10], '\nerror \n', error[:10])
            # print("random number \n", random_array[:10])
            # print(gamma_j[:10])
            # print("synthetic rate \n", synthetic_rate[:10])
            # print(f"printed for {i}")

            #Run efsearch on the synthetic light curve
            efsearch_cmd = f'efsearch {output_fits} window=- sepoch=INDEF dper=9.801 nphase=64 nbint=INDEF dres=1e-6 nper=1024 plot=no outfile={output_folder}/{obs_id}/testing_efsearch.fes'
            try:
                subprocess.run(efsearch_cmd, shell=True, check=True)

                # Open the .fes FITS file and extract the best-fit period
                with fits.open(f'{output_folder}/{obs_id}/testing_efsearch.fes') as efsearch_fits:
                    results_data = efsearch_fits['RESULTS'].data
                    max_chisq_index = np.argmax(results_data['CHISQRD1'])
                    best_fit_period = results_data['PERIOD'][max_chisq_index]
                    chisq_max=results_data['CHISQRD1'][max_chisq_index]

                # Write the light curve name and best-fit period to the individual output file
                with open(best_fit_period_file, 'a') as f_out:
                    f_out.write(f"{output_fits}\t{best_fit_period}\t{chisq_max}\n")
                print(f"Processed {output_fits}, Best-fit period: {best_fit_period}")

            except subprocess.CalledProcessError as e:
                print(f"Error processing {output_fits}: {e}")

            # Delete the synthetic FITS file to save space
            os.remove(output_fits)
            print(f"Deleted synthetic light curve {i+1} to save space.")

def read_light_curve_paths(base_dir):
    light_curve_paths = []
    
    # Loop through each obs_id in the base directory
    for obs_id in os.listdir(base_dir):
        obs_dir = os.path.join(base_dir, obs_id)

        if os.path.isdir(obs_dir):  # Check if it's a directory
            input_fits_path = os.path.join(obs_dir, 'xti', 'event_cl', f'ni{obs_id}_cl_night_barycorrmpu7_sr_night_0.01.lc')

            if os.path.isfile(input_fits_path):  # Check if the light curve file exists
                light_curve_paths.append(input_fits_path)

    return light_curve_paths


In [ ]:
if __name__ == "__main__":
    # Base directory containing observation ID folders
    base_dir = './'
    output_folder = './bootstrap_error/'

    # Number of synthetic light curves to generate
    num_synthetic_curves = 1000
    # Read light curve paths
    light_curve_paths = read_light_curve_paths(base_dir)

        
    # Generate synthetic light curves for all light curve files found and run efsearch
    for path in light_curve_paths:
        generate_synthetic_light_curves(path, output_folder, num_synthetic_curves)

    print("Finished processing all light curves.")

 
efsearch 1.1 (xronos6.0)
 
 Series 1 file    1:./bootstrap_error/6050390244/synthetic_light_curve_1.fits
 
 Selected FITS extensions: 1 - RATE TABLE;
 
 Source ............ Swift_J0243.6+61    Start Time (d) .... 20130 05:44:42.284
 FITS Extension ....  1 - `RATE      `   Stop Time (d) ..... 20130 18:13:22.084
 No. of Rows .......       183980        Bin Time (s) ......   0.1000E-01
 Right Ascension ... 40.9186             Internal time sys.. Converted to TJD
 Declination ....... 61.4323             Experiment ........ NICER    XTI
 
 Corrections applied: Vignetting - No ; Deadtime - No ; Bkgd - No ; Clock - Yes
 
 Selected Columns:  1- Time;  2- Y-axis;  3- Y-error;  4- Fractional exposure;
 
 File contains binned data.
 
 
 Expected Start ... 20130.23937828703  (days)       5:44:42:284  (h:m:s:ms)
 Expected Stop .... 20130.75928337963  (days)      18:13:22: 84  (h:m:s:ms)
 
 Default Epoch is:  20130.00000
Type INDEF to accept the default value
 Epoch format is days.
 Period format 